In [1]:
import sys
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats
from feature_engineering import word_overlap_features
from Dataset import DataSet
from generate_test_splits import kfold_split, get_stances_for_folds
from score import report_score, LABELS, score_submission
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd

number_folds = 2

In [2]:
d = DataSet()
len(d.articles)

Reading dataset
Total stances: 49972
Total bodies: 1683


1683

In [3]:
folds,hold_out = kfold_split(d,training=1.0,n_folds=2)
fold_stances, hold_out_stances = get_stances_for_folds(d,folds,hold_out)

In [4]:
print(fold_stances[0][0])
#LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
lookup = dict()
lookup['agree'] = 0
lookup['disagree'] = 1
lookup['discuss'] = 0
lookup['unrelated'] = 1

{'Headline': "Christian Bale passes on role of Steve Jobs, actor reportedly felt he wasn't right for part", 'Body ID': 137, 'Stance': 'unrelated'}


In [5]:
train_data = []
for fold_id in range(len(fold_stances)):
    xdd = []
    for i in range(len(fold_stances[fold_id])):
        article_text = d.articles[fold_stances[fold_id][i]['Body ID']]
        new_list = [fold_stances[fold_id][i]['Headline'] , article_text,int(lookup[fold_stances[fold_id][i]['Stance']])]
        xdd.append(new_list)
    train_data.append(xdd)

In [6]:
train_data[0][0]

["Christian Bale passes on role of Steve Jobs, actor reportedly felt he wasn't right for part",
 '30-year-old Moscow resident was hospitalized with wounds very intimate nature. As it became known LifeNews, in the hands of doctors, the man complained that his casual acquaintance opoila in the sauna, and then gently held his castration operation. And actions criminals were executed with surgical precision - woman sewed all the smallest blood vessels.\n\n\n\n- I met a girl at the bar, and then we went to the sauna for a taxi. I remember that I had a beer, and more do not remember, - said Roman law enforcement officials.\n\nAt the hospital, the man hours spent in intensive care, doctors were forced to spend a second operation on the testicles.\n\n- I thought I had just cut, but at the hospital they told me that I removed the testicles - told Roman about his sad adventures.\n\n- The patient had literally jewelry operation - he cut out eggs and ligated vessels. We can say that the transactio

In [7]:
for iters in range(len(fold_stances)):
    train_df = pd.DataFrame(train_data[iters])
    train_df.columns = ["text_a", "text_b", "labels"]
    
    eval_df = pd.DataFrame(train_data[1-iters])
    eval_df.columns = ["text_a", "text_b", "labels"]
    eval_labels = eval_df["labels"].tolist()
    eval_df.drop("labels" , axis = 1)
    #print(train_df)
    print(eval_df)
    model_args = ClassificationArgs(num_train_epochs=1)
    model = ClassificationModel("roberta", "roberta-base",use_cuda=False)
    
    # Train the model
    model.train_model(train_df)
    
    # Evaluate the model
    result, model_outputs, wrong_predictions = model.eval_model(eval_df)
    
    predicted = [LABELS[int(a)] for a in model_outputs]
    actual = [LABELS[int(a)] for a in eval_labels]

    fold_score, _ = score_submission(actual, predicted)
    max_fold_score, _ = score_submission(actual, actual)

    score = fold_score/max_fold_score

    print("Score for fold "+ str(iters) + " was - " + str(score))

                                                  text_a  \
0      Hundreds of Palestinians flee floods in Gaza a...   
1      HBO and Apple in Talks for $15/Month Apple TV ...   
2      Spider burrowed through tourist's stomach and ...   
3      Accused Boston Marathon Bomber Severely Injure...   
4          British Aid Worker Confirmed Murdered By ISIS   
...                                                  ...   
19782  Nicaraguan commission on possible meteorite cr...   
19783     Would-be rapist has penis severed by angry mob   
19784  'Banksy' Reacts To Paris Attack With Poignant ...   
19785  Indian 'Rapist' Castrated By Angry Mob Who Hac...   
19786  Brian Williams slams social media for speculat...   

                                                  text_b  labels  
0      Hundreds of Palestinians were evacuated from t...       0  
1      (Reuters) - A Canadian soldier was shot at the...       1  
2      Fear not arachnophobes, the story of Bunbury's...       1  
3      A Br

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

KeyboardInterrupt: 